# Run Bot By predict Left & Right motor values

This notebook is to run the robot using left and right motor values predicted from model.  
**Model File**       > HDF5 (.h5)  
**Model Output**     > Left/right motor values  
**Folder Structure** >
<pre style="font-size: 10.0pt; font-family: Arial; line-height: 1; letter-spacing: 0.0pt;" >
<b>ITI110_Project/</b>
|__ <b>data/</b>
|  |__ <b>raw/</b>
|__ <b>models/</b>
|  |__ <b>run/</b>
|    |__<b>{Your Model Name}.h5</b>  
|__ <b>run_bot_hdf5_pred_motors.ipynb</b>
</pre>

# Import Libraries

In [1]:
import traitlets
from IPython.display import display
import ipywidgets
from jetbot import Camera, bgr8_to_jpeg, Robot
import numpy as np
import tensorflow as tf
from tensorflow.keras import models
import time, PIL, io, os, pathlib, cv2

In [2]:
print(tf.__version__)
tf.enable_eager_execution()

1.14.0


# Load Model

Load Keras HDF5 file (.h5) into memory for inference

In [4]:
MODEL_DIR = os.path.join(os.getcwd(),'models')
MODEL_PATH = os.path.join(MODEL_DIR, 'Pilotnet_CAT_YUV_50E_TF114_model.tflite') ## Change here if you have different name

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=MODEL_PATH)
interpreter.allocate_tensors()
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(f'Loaded Model: {MODEL_PATH}')

Loaded Model: /home/jetbot/ITI110_Project/models/Pilotnet_CAT_YUV_50E_TF114_model.tflite


# Define Preprocess()

The function is use to pre-process input image into desired format required by model. This function must same with the one using in training model.

In [6]:
def convertToOneHot(left,right):
    if left == right:
        return [0,1,0]
    elif left > right: # Turn Right
        return [0,0,1]
    elif right > left: # Turn Left
        return [1,0,0]

## This function have to deploy to jetbot
def preprocess(image, h=224, w=224, crop_top=0.0, color=False): 

    """
    The function is to preprocess image before feeding into model. 
    Current feature include resize image based on h and w, vertical cropping, and change color.
    
    :param image: input imag
    :param h: target height of output image in pixel. Default to 224
    :param w: target width of output image in pixel. Default to 224
    :param crop_top: Determine whether to crop the top part of the image. use ratio value. e.g. 0.5 mean crop top half of image. Default to 0
    :param color: cv2 color conversion. Put False if want to keep original color. Default to cv2.COLOR_RGB2YUV. Possible values [cv2.COLOR_BGR2GRAY,cv2.COLOR_RGB2YUV,cv2.COLOR_RGB2GRAY,cv2.COLOR_RGB2GRAY, cv2.COLOR_BGR2RGB, cv2.COLOR_RGB2BGR]
    :return normalized and processed image
    
    """

    height, _, _ = image.shape
    image = image[int(height*crop_top):,:,:]  # remove top half of the image, as it is not relevant for lane following
    if color:
      image = cv2.cvtColor(image, color) 
    image = cv2.GaussianBlur(image, (3,3), 0)
    image = cv2.resize(image, (h,w)) # input image size (200,66) Nvidia model
    image = image / 255 # normalizing

    return image
  
def process_prediction(prediction):
    predicted_index = np.argmax(prediction)
    result = np.zeros(len(prediction))
    result[predicted_index] = 1
    return result

Setup Preprocess() parameters

In [ ]:
IMG_H = 200 ## Change Here
IMG_W = 66 ## Change Here
T_CROP = 0 ## Change Here
COLOR_CONV = cv2.COLOR_BGR2YUV ## Change Here

# Define Execute()

This function use to continuously capture image, predict value, and run Robot's steering  
**To Complete**

In [9]:
angle = 0.0
angle_last = 0.0
SPEED = 0.2
def execute(change):
    global angle, angle_last
        
    speed_slider.value = SPEED
    
    image = change['new']
    image = preprocess(image,IMG_H,IMG_W,T_CROP,COLOR_CONV)
    h, w, c = image.shape
    image = image.reshape(1,h,w,c)
    image = np.array(image, dtype=np.float32) ## Change dtype based on input details print out from cell above
    interpreter.set_tensor(input_details[0]['index'], image)
    interpreter.invoke()
    left_right = process_prediction(interpreter.get_tensor(output_details[0]['index'])[0]).astype('int32').tolist()
    
    #steering_slider.value = pid + steering_bias_slider.value
    robot.left_motor.value  = speed_slider.value if left_right[0] == 1 else (speed_slider.value * 0.5)
    robot.right_motor.value = speed_slider.value if left_right[2] == 1 else (speed_slider.value * 0.5)

# Start Robot

Initialize camera and robot instances  
Note that sometimes when we restart the kernels, the camera might not be re-initialized. We might need to run the following in the jetbot terminal.
```bash
$ sudo systemctl restart nvargus-daemon
```

In [10]:
camera = Camera.instance(width=224, height=224)
robot = Robot()

Create Live view and config widgets

In [11]:
image_widget = ipywidgets.Image(format='jpeg', width=224, height=224)
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

Run code below to execute camera value to execute() function

In [12]:
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

Execute code below for attach camera to trailets. It will start moving the robot by continuous feed image

In [13]:
display(ipywidgets.HBox([image_widget,y_slider, speed_slider]))
display(x_slider, steering_slider)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

In [15]:
execute({'new': camera.value}) # we call the function once to intialize
#time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

[0. 0. 1.]


In [16]:
camera.observe(execute, names='value')
#time.sleep(0.1)  # add a small sleep to make sure frames have finished processing
#robot.stop()

[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[1. 0. 0.]
[0. 0. 1.]
[0. 0. 1.]
[0. 1. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]

# Stop Robot

In [39]:
camera.unobserve(execute, names='value')

ValueError: list.remove(x): x not in list

In [38]:
camera.unobserve(execute, names='value')
time.sleep(0.1)  # add a small sleep to make sure frames have finished processing
robot.stop()

ValueError: list.remove(x): x not in list